In [145]:
from EduData import get_data

get_data("assistment-2009-2010-skill", "data")

downloader, INFO http://base.ustc.edu.cn/data/ASSISTment/2009_skill_builder_data_corrected.zip is saved as data/2009_skill_builder_data_corrected.zip
downloader, INFO file existed, skipped


'data/2009_skill_builder_data_corrected'

In [146]:
import random
import pandas as pd
import tqdm
import numpy as np

data = pd.read_csv(
    'data/2009_skill_builder_data_corrected/skill_builder_data_corrected.csv',
    usecols=['order_id', 'user_id', 'sequence_id', 'skill_id', 'correct'],encoding="utf-8"
).dropna(subset=['skill_id'])

In [163]:
data = pd.read_csv(
    'data/2009_skill_builder_data_corrected/skill_builder_data_corrected.csv',
    usecols=['order_id', 'user_id', 'sequence_id', 'skill_id', 'correct','problem_id'],encoding="utf-8"
).dropna(subset=['skill_id','problem_id'])

In [148]:
len(data.problem_id.unique())

17751

In [149]:
len(data.groupby(['problem_id','skill_id']))

21246

In [150]:
raw_skill = data.skill_id.unique().tolist()
raw_problem = data.problem_id.unique().tolist()

num_skill = len(raw_question)
num_problem = len(raw_problem)

# question id from 0 to (num_skill - 1)
skills = { p: i for i, p in enumerate(raw_skill) }
problem = { p: i for i, p in enumerate(raw_problem) }

print("number of skills: %d" % num_skill)
print("number of problems: %d" % num_problem)

number of skills: 123
number of problems: 17751


In [151]:
p_embedding = np.zeros((num_problem,num_skill))

In [152]:
for index, row in data.iterrows():
    p_id = problem[row['problem_id']]
    s_id = skills[row['skill_id']]
    p_embedding[p_id][s_id] = 1

In [153]:
p_embedding.shape

(17751, 123)

In [154]:
def parse_all_seq(students):
    all_sequences = []
    for student_id in tqdm.tqdm(students, 'parse student sequence:\t'):
        student_sequence = parse_student_seq(data[data.user_id == student_id])
        all_sequences.extend([student_sequence])
    return all_sequences


def parse_student_seq(student):
    seq = student.sort_values('order_id')
    s = [skills[q] for q in seq.skill_id.tolist()]
    p = [problem[q] for q in seq.problem_id.tolist()]
    a = seq.correct.tolist()
    return s,p,a


# [(question_sequence_0, answer_sequence_0), ..., (question_sequence_n, answer_sequence_n)]
sequences = parse_all_seq(data.user_id.unique())

parse student sequence:	: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4163/4163 [00:01<00:00, 3006.96it/s]


In [155]:
len(sequences)

4163

In [156]:
i = 0
for s,p,a in sequences:
    print(len(q),len(a),len(s))
    print(s)
    print(p)
    print(a)
    i += 1
    if i > 3:
        break

311 742 742
[80, 80, 80, 80, 80, 82, 81, 82, 42, 80, 37, 79, 81, 81, 79, 79, 36, 17, 81, 80, 34, 32, 81, 96, 96, 96, 96, 96, 37, 55, 55, 37, 80, 82, 82, 37, 37, 37, 37, 37, 32, 30, 30, 30, 30, 30, 30, 30, 30, 22, 51, 44, 78, 38, 81, 81, 56, 47, 48, 48, 48, 48, 48, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 79, 81, 81, 79, 41, 41, 41, 41, 42, 42, 81, 32, 33, 32, 32, 33, 33, 33, 33, 33, 33, 34, 34, 35, 46, 34, 46, 34, 35, 35, 46, 34, 46, 35, 34, 35, 46, 34, 34, 46, 35, 35, 34, 46, 96, 96, 97, 97, 46, 46, 46, 46, 46, 46, 46, 44, 54, 49, 49, 49, 49, 36, 50, 52, 52, 52, 52, 52, 49, 49, 49, 49, 49, 56, 49, 49, 49, 49, 49, 44, 1, 44, 44, 1, 48, 48, 48, 48, 46, 39, 46, 39, 46, 39, 46, 46, 39, 39, 46, 46, 39, 39, 46, 46, 39, 39, 46, 82, 79, 51, 51, 53, 53, 53, 53, 53, 53, 46, 34, 35, 46, 35, 34, 46, 35, 34, 46, 35, 34, 35, 46, 34, 34, 35, 46, 34, 35, 46, 46, 35, 34, 53, 55, 37, 55, 37, 79, 80, 82, 82, 37, 37, 37, 37, 37, 32, 30, 30, 30, 30, 30, 30, 30, 30, 30, 22, 44, 78, 38, 81, 81, 56, 47, 48, 4

In [157]:
def train_test_split(data, train_size=.7, shuffle=True):
    if shuffle:
        random.shuffle(data)
    boundary = round(len(data) * train_size)
    return data[: boundary], data[boundary:]


train_sequences, test_sequences = train_test_split(sequences)

In [158]:
len(train_sequences[0])

3

In [159]:
num_skill

123

In [160]:
def sequences2tl(sequences, trgpath):
    with open(trgpath, 'a', encoding='utf8') as f:
        for seq in tqdm.tqdm(sequences, 'write into file: '):
            s,p,a= seq
            seq_len = len(s)
            f.write(str(seq_len) + '\n')
            f.write(','.join([str(x) for x in s]) + '\n')
            f.write(','.join([str(x) for x in p]) + '\n')
            f.write(','.join([str(x) for x in a]) + '\n')

# save triple line format for other tasks
sequences2tl(train_sequences, 'data/2009_skill_builder_data_corrected/train.txt')
sequences2tl(test_sequences, 'data/2009_skill_builder_data_corrected/test.txt')

write into file: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1249/1249 [00:00<00:00, 45051.17it/s]


In [118]:
MAX_STEP = 100

def encode_onehot(sequences, max_step):
    skill_result = []
    problem_result = []
    problem_embedding_result = []
    answer_result = []
    real_result = []
    for s, p, a in tqdm.tqdm(sequences, 'convert to one-hot format: '):
        s_len = len(s)
        left = MAX_STEP - s_len % MAX_STEP
        s = s + [0]*left
        p = p + [0]*left
        a = a + [0]*left
        last_real_len = s_len % MAX_STEP
        first_real_len = [MAX_STEP] * ((s_len - last_real_len) // MAX_STEP)
        real_len = first_real_len + [last_real_len]
        num = (s_len+left)//MAX_STEP
        skill_seq = []
        problem_seq = []
        answer_seq = []
        problem_embedding_seq = []
        for i in range(num):
            skill_seq.append(s[i*MAX_STEP:(i+1)*MAX_STEP])
            problem_seq.append(p[i*MAX_STEP:(i+1)*MAX_STEP])
            answer_seq.append(a[i*MAX_STEP:(i+1)*MAX_STEP])
            problem_embedding = []
            for index in s[i*MAX_STEP:(i+1)*MAX_STEP]:
                one_hot = [0]*num_skill
                if index != -1:
                    one_hot[index] = 1
                problem_embedding.append(one_hot)
            problem_embedding_seq.append(problem_embedding)
        skill_result.extend(skill_seq)
        problem_result.extend(problem_seq)
        answer_result.extend(answer_seq)
        problem_embedding_result.extend(problem_embedding_seq)
        real_result.extend(real_len)
    return np.array(skill_result),np.array(problem_result),np.array(answer_result),np.array(problem_embedding_result),np.array(real_result)


# reduce the amount of data for example running faster
percentage = 1.0
train_data = encode_onehot(train_sequences[: int(len(train_sequences) * percentage)], MAX_STEP)
test_data = encode_onehot(test_sequences[: int(len(test_sequences) * percentage)], MAX_STEP)

convert to one-hot format: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1249/1249 [00:00<00:00, 8052.05it/s]


In [161]:
len(train_data[-2])

4485

In [120]:
train_data[-2].shape

(4485, 100, 123)

In [94]:
train_data[3].shape

(4485, 100, 123)

In [122]:
np.savez('ass9.npz',
        y=train_data[2],
         skill=train_data[0],
         problem=train_data[1],
         problem_embedding=p_embedding,
         real_len=train_data[4],
         skill_num=num_skill,
         problem_num=num_problem
        )

In [126]:
np.save('pro_skill_data',p_embedding)

In [127]:
from scipy import sparse

In [129]:
sparse.save_npz('pro_skill_sparse.npz',sparse.coo_matrix(p_embedding))

In [ ]:
difficult = np.zeros((num_problem,7))
for index, row in data.iterrows():
    p_id = problem[row['problem_id']]
    s_id = skills[row['skill_id']]
    difficult[p_id][-1] += row['correct']
    difficult[p_id][-2] += 1

In [ ]:
difficult[:,-1] = difficult[:,-1] / difficult[:,-2]

In [ ]:
difficult[:,-2]=0

In [ ]:
difficult[:,-1]

In [ ]:
sparse.save_npz('pro_feat.npz',sparse.coo_matrix(difficult))

In [168]:
difficult[:,-1] = difficult[:,-1] / difficult[:,-2]

In [170]:
difficult[:,-2]=0

In [174]:
difficult[:,-1]

array([0.76470588, 0.875     , 0.65517241, ..., 1.        , 1.        ,
       1.        ])

In [175]:
sparse.save_npz('pro_feat.npz',sparse.coo_matrix(difficult))